## 1. Installation et imports

In [1]:
# Installation des bibliothèques nécessaires
!pip install -q transformers datasets peft accelerate torch bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 32.6 MB/s eta 0:00:00:00:0100:01


In [2]:
# # === GOOGLE COLAB ONLY ===
# from google.colab import drive
# import os
# import shutil

# # Monter Google Drive
# drive.mount('/content/drive')

# # Chemin vers votre fichier ZIP sur Google Drive
# # ⚠️ Adaptez ce chemin selon l'emplacement exact de votre fichier
# DRIVE_ZIP_PATH = "/content/drive/MyDrive/output_dir.zip"  # <- Modifiez ici !

# ZIP_FILENAME = "output_dir.zip"

# if os.path.exists(DRIVE_ZIP_PATH):
#     shutil.copy(DRIVE_ZIP_PATH, ZIP_FILENAME)
#     print(f"✓ Fichier copié depuis Google Drive : {DRIVE_ZIP_PATH}")
#     print(f" → {ZIP_FILENAME} ({os.path.getsize(ZIP_FILENAME) / 1e6:.2f} MB)")
# else:
#     print(f"⚠️ Fichier non trouvé : {DRIVE_ZIP_PATH}")
#     print("   Vérifiez le chemin et assurez-vous que le Drive est bien monté.")

In [3]:
# === Téléchargement direct depuis Google Drive avec gdown (Kaggle) ===
import os

# ⚠️ Remplacez par l'ID de votre fichier ZIP
FILE_ID = "13EnWzFS99_8gPSUr5n7e5qeaZWM6t-5s"  # Ex: 1A2B3C4D5E6F7G8H9I0J

ZIP_FILENAME = "output_dir.zip"

# gdown est déjà disponible sur Kaggle (pas besoin d'install)
!gdown --id {FILE_ID} -O {ZIP_FILENAME}

if os.path.exists(ZIP_FILENAME):
    print(f"✓ Téléchargement réussi : {ZIP_FILENAME} ({os.path.getsize(ZIP_FILENAME) / 1e6:.2f} MB)")
else:
    print("⚠️ Échec du téléchargement. Vérifiez l'ID et les permissions de partage.")

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=13EnWzFS99_8gPSUr5n7e5qeaZWM6t-5s
From (redirected): https://drive.google.com/uc?id=13EnWzFS99_8gPSUr5n7e5qeaZWM6t-5s&confirm=t&uuid=06c66a4b-346e-45e1-bb2e-36ec9fe79a5f
To: /kaggle/working/output_dir.zip
100%|███████████████████████████████████████| 1.54G/1.54G [00:11<00:00, 133MB/s]
✓ Téléchargement réussi : output_dir.zip (1543.61 MB)


In [4]:
import os
import zipfile
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig  # Pour la quantification 4-bit
)
from peft import PeftModel, PeftConfig
from datasets import load_dataset
from tqdm.auto import tqdm
import gc  # Pour libérer la mémoire

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

2025-12-22 23:39:25.558797: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766446765.783167      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766446765.848565      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766446766.378574      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766446766.378604      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766446766.378606      55 computation_placer.cc:177] computation placer alr

PyTorch version: 2.8.0+cu126
CUDA available: True
CUDA device: Tesla T4
GPU Memory: 15.83 GB


## 2. Configuration

In [7]:
# Chemins et paramètres
ROOT = "/kaggle/working/"
LORA_ZIP_PATH = ROOT+"output_dir.zip"
LORA_DIR = ROOT+"output_dir"
TEACHER_MODEL = "mistralai/Mistral-7B-v0.1"
STUDENT_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# =============================================================================
# PARAMÈTRES DE DISTILLATION AMÉLIORÉS
# =============================================================================
TEMPERATURE = 4.0           # Plus élevée pour adoucir les distributions (était 2.0)
ALPHA_CE = 0.5              # Poids de la loss supervisée (cross-entropy sur hard labels)
ALPHA_KL = 0.5              # Poids de la loss de distillation (KL sur soft labels)
MAX_LENGTH = 256            # Augmenté pour capturer plus de contexte (était 128)
BATCH_SIZE = 1              # Batch size minimal (contrainte mémoire)
LEARNING_RATE = 5e-6        # Réduit pour éviter le catastrophic forgetting (était 2e-5)
NUM_EPOCHS = 3
GRADIENT_ACCUMULATION_STEPS = 16
WARMUP_RATIO = 0.1          # 10% de warmup

# Chemins de sauvegarde
OUTPUT_MODEL_DIR = "distilled_tinyllama"
OFFLOAD_FOLDER = "offload_teacher"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"\n{'='*60}")
print("CONFIGURATION AMÉLIORÉE POUR LA DISTILLATION")
print(f"{'='*60}")
print(f"  - Temperature: {TEMPERATURE} (distributions adoucies)")
print(f"  - Alpha CE (hard labels): {ALPHA_CE}")
print(f"  - Alpha KL (soft labels): {ALPHA_KL}")
print(f"  - Learning rate: {LEARNING_RATE} (conservateur)")
print(f"  - MAX_LENGTH: {MAX_LENGTH}")
print(f"  - Effective batch size: {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")

Using device: cuda

CONFIGURATION AMÉLIORÉE POUR LA DISTILLATION
  - Temperature: 4.0 (distributions adoucies)
  - Alpha CE (hard labels): 0.5
  - Alpha KL (soft labels): 0.5
  - Learning rate: 5e-06 (conservateur)
  - MAX_LENGTH: 256
  - Effective batch size: 16


## 3. Extraction des adaptateurs LoRA

In [ ]:
# Créer le répertoire output_dir/ s'il n'existe pas
if not os.path.exists(LORA_DIR):
    print(f"Création du répertoire {LORA_DIR}/...")
    os.makedirs(LORA_DIR, exist_ok=True)
    print(f"Répertoire {LORA_DIR}/ créé")
    
    if os.path.exists(LORA_ZIP_PATH):
        print(f"\nExtraction de {LORA_ZIP_PATH} dans {LORA_DIR}/...")
        with zipfile.ZipFile(LORA_ZIP_PATH, 'r') as zip_ref:
            zip_ref.extractall(LORA_DIR)
        print(f"Extraction terminée")
    else:
        print(f"{LORA_ZIP_PATH} introuvable")
else:
    print(f"Le répertoire {LORA_DIR}/ existe déjà")

if os.path.exists(LORA_DIR) and os.listdir(LORA_DIR):
    print(f"\nContenu de {LORA_DIR}/:")
    for item in os.listdir(LORA_DIR):
        print(f"  - {item}")
else:
    print(f"\nLe répertoire {LORA_DIR}/ est vide. Placez-y les fichiers suivants:")
    print("  - adapter_config.json")
    print("  - adapter_model.bin (ou adapter_model.safetensors)")
    print("  - tokenizer_config.json, tokenizer.json, etc.")

✓ Le répertoire /kaggle/working/output_dir/ existe déjà

Contenu de /kaggle/working/output_dir/:
  - special_tokens_map.json
  - tokenizer.model
  - tokenizer.json
  - adapter_model.safetensors
  - checkpoint-1657
  - adapter_config.json
  - README.md
  - tokenizer_config.json
  - all_results.json
  - train_results.json
  - eval_results.json


## 4. Chargement du dataset

Chargement du dataset d'instructions utilisé lors du fine-tuning QLoRA. Les labels humains sont ignorés : seules les instructions servent d'entrée pour le teacher.

In [9]:
from datasets import load_dataset, concatenate_datasets

# === CHOIX DU OU DES DATASETS ===

# 1. Dataset principal de haute qualité 
# Option A : OpenHermes-2.5 (très bon pour instruction + raisonnement, réponses par Mistral/GPT-4)
# dataset_hermes = load_dataset("teknium/OpenHermes-2.5", split="train")
dataset_hermes = load_dataset("tatsu-lab/alpaca", split="train")


# Option B : UltraChat-200k (très diversifié, bonnes réponses)
# dataset_ultra = load_dataset("HuggingFaceH4/ultrachat_200k", split="train_sft")

# Option C : Mix pour plus de volume et diversité
# dataset_alpaca = load_dataset("tatsu-lab/alpaca", split="train")
# dataset_gsm8k = load_dataset("gsm8k", "main", split="train")  # Pour le raisonnement math

# === FOCUS SUR LE RAISONNEMENT MATH (critique pour ton cas) ===
# Ajoute explicitement GSM8K avec chain-of-thought (version CoT)
dataset_gsm8k = load_dataset("openai/gsm8k", "main", split="train")
# Ou mieux : une version avec réponses détaillées
# dataset_gsm8k = load_dataset("reasoning-machines/gsm8k-cot", split="train")

# === CONSTRUCTION DU DATASET FINAL ===
# Prends tout OpenHermes + tout GSM8K (excellent équilibre qualité/raisonnement)
dataset = concatenate_datasets([dataset_hermes, dataset_gsm8k])

# Option : sous-échantillonnage pour tests rapides (mais pas trop petit !)
# Garde au moins 10k-20k exemples pour une démo sérieuse
dataset = dataset.select(range(min(17000, len(dataset))))  # Max 20k pour ne pas exploser le temps

# Mélange bien
dataset = dataset.shuffle(seed=42)

print(f"Nombre d'exemples chargés: {len(dataset):,}")
print(f"Sources: OpenHermes-2.5 + GSM8K (CoT)")
print("\nExemples aléatoires:")

for i in [0, 1, 2]:
    print(f"\n--- Exemple {i+1} ---")
    if 'conversations' in dataset[i]:  # Format OpenHermes
        for msg in dataset[i]['conversations']:
            print(f"{msg['from'].upper()}: {msg['value']}")
    elif 'question' in dataset[i]:  # GSM8K
        print("QUESTION:", dataset[i]['question'])
        print("ANSWER (CoT):", dataset[i]['answer'])
    else:  # Alpaca-like
        print("INSTRUCTION:", dataset[i].get('instruction', dataset[i].get('question')))
        print("OUTPUT:", dataset[i].get('output', dataset[i].get('answer')))

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Nombre d'exemples chargés: 17,000
Sources: OpenHermes-2.5 + GSM8K (CoT)

Exemples aléatoires:

--- Exemple 1 ---
QUESTION: None
ANSWER (CoT): None

--- Exemple 2 ---
QUESTION: None
ANSWER (CoT): None

--- Exemple 3 ---
QUESTION: None
ANSWER (CoT): None


In [10]:
# Fonction pour formater les prompts
def format_instruction(example):
    instruction = example['instruction']
    input_text = example.get('input', '')
    
    if input_text:
        prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n"
    else:
        prompt = f"### Instruction:\n{instruction}\n\n### Response:\n"
    
    return {"text": prompt}

# Appliquer le formatage
dataset = dataset.map(format_instruction)
print(f"\nExemple formaté:\n{dataset[0]['text'][:200]}...")

Map:   0%|          | 0/17000 [00:00<?, ? examples/s]


Exemple formaté:
### Instruction:
Write a short sentence about the benefits of going to the beach.

### Input:
(noinput)

### Response:
...


## 5. Chargement du modèle Teacher (Mistral-7B + LoRA)

In [ ]:
print("Chargement du modèle teacher (Mistral-7B) en 4-bit...")

os.makedirs(OFFLOAD_FOLDER, exist_ok=True)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

print("  Chargement du modèle de base en 4-bit...")
teacher_base = AutoModelForCausalLM.from_pretrained(
    TEACHER_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True
)

print(f"  Chargement des adaptateurs LoRA depuis {LORA_DIR}...")
teacher_model = PeftModel.from_pretrained(
    teacher_base,
    LORA_DIR,
    is_trainable=False
)

for param in teacher_model.parameters():
    param.requires_grad = False

teacher_model.eval()

gc.collect()
torch.cuda.empty_cache()

print("Modèle teacher chargé en 4-bit et frozen")
print(f"  Mémoire GPU utilisée: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

teacher_tokenizer = AutoTokenizer.from_pretrained(LORA_DIR)
if teacher_tokenizer.pad_token is None:
    teacher_tokenizer.pad_token = teacher_tokenizer.eos_token
print("Tokenizer chargé")

Chargement du modèle teacher (Mistral-7B) en 4-bit...
  → Chargement du modèle de base en 4-bit...


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

  → Chargement des adaptateurs LoRA depuis /kaggle/working/output_dir...


/usr/local/lib/python3.12/dist-packages/peft/config.py:165: UserWarning: Unexpected keyword arguments ['alora_invocation_tokens', 'arrow_config', 'ensure_weight_tying', 'peft_version'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


✓ Modèle teacher chargé en 4-bit et frozen
  Mémoire GPU utilisée: 2.00 GB
✓ Tokenizer chargé


## 6. Chargement du modèle Student (TinyLLaMA-1.1B)

In [ ]:
print("Chargement du modèle student (TinyLLaMA-1.1B)...")

gc.collect()
torch.cuda.empty_cache()

student_model = AutoModelForCausalLM.from_pretrained(
    STUDENT_MODEL,
    dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True
)

student_model.gradient_checkpointing_enable()

student_model.train()
print("Modèle student chargé (trainable + gradient checkpointing)")
print(f"  Mémoire GPU totale utilisée: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

student_tokenizer = AutoTokenizer.from_pretrained(STUDENT_MODEL)
if student_tokenizer.pad_token is None:
    student_tokenizer.pad_token = student_tokenizer.eos_token
print("Tokenizer student chargé")

tokenizer = teacher_tokenizer

Chargement du modèle student (TinyLLaMA-1.1B)...


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✓ Modèle student chargé (trainable + gradient checkpointing)
  Mémoire GPU totale utilisée: 4.20 GB


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

✓ Tokenizer student chargé


## 7. Préparation du dataset pour la distillation

In [13]:
# Tokenizer les données
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=MAX_LENGTH,
        return_tensors=None
    )

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset.column_names
)

tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
print(f"Dataset tokenisé: {len(tokenized_dataset)} exemples")

Map:   0%|          | 0/17000 [00:00<?, ? examples/s]

Dataset tokenisé: 17000 exemples


## 8. Définition de la loss de distillation

In [ ]:
class DistillationLoss(nn.Module):
    """
    Loss de distillation basée sur la divergence KL entre les logits du teacher et du student.
    La température T adoucit les distributions de probabilités.
    Le facteur T^2 compense la normalisation lors de l'utilisation de softmax avec température.
    """
    def __init__(self, temperature=2.0):
        super().__init__()
        self.temperature = temperature
        self.kl_div = nn.KLDivLoss(reduction='batchmean')
    
    def forward(self, student_logits, teacher_logits, attention_mask=None):
        student_log_probs = F.log_softmax(student_logits / self.temperature, dim=-1)
        teacher_probs = F.softmax(teacher_logits / self.temperature, dim=-1)
        
        kl_loss = self.kl_div(student_log_probs, teacher_probs)
        distillation_loss = kl_loss * (self.temperature ** 2)
        
        return distillation_loss

distillation_criterion = DistillationLoss(temperature=TEMPERATURE)
print(f"Loss de distillation initialisée (T={TEMPERATURE})")

✓ Loss de distillation initialisée (T=4.0)


## 9. Configuration de l'entraînement

In [ ]:
# DataLoader
train_dataloader = DataLoader(
    tokenized_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

try:
    import bitsandbytes as bnb
    optimizer = bnb.optim.Adam8bit(
        student_model.parameters(),
        lr=LEARNING_RATE
    )
    print("Optimiseur: 8-bit Adam (économie mémoire ~75%)")
except ImportError:
    optimizer = torch.optim.AdamW(
        student_model.parameters(),
        lr=LEARNING_RATE
    )
    print("Optimiseur: AdamW standard (bitsandbytes non disponible)")

from transformers import get_linear_schedule_with_warmup

total_steps = len(train_dataloader) * NUM_EPOCHS // GRADIENT_ACCUMULATION_STEPS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=total_steps // 10,
    num_training_steps=total_steps
)

print(f"\nConfiguration:")
print(f"  - Batch size: {BATCH_SIZE}")
print(f"  - Gradient accumulation: {GRADIENT_ACCUMULATION_STEPS}")
print(f"  - Effective batch size: {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"  - Learning rate: {LEARNING_RATE}")
print(f"  - Total steps: {total_steps}")
print(f"  - Mémoire GPU utilisée: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

✓ Optimiseur: 8-bit Adam (économie mémoire ~75%)

Configuration:
  - Batch size: 1
  - Gradient accumulation: 16
  - Effective batch size: 16
  - Learning rate: 5e-06
  - Total steps: 3187
  - Mémoire GPU utilisée: 4.20 GB


## 10. Boucle d'entraînement (Distillation)

In [ ]:
print("\n" + "="*60)
print("DÉBUT DE LA DISTILLATION")
print("="*60 + "\n")

gc.collect()
torch.cuda.empty_cache()
print(f"Mémoire GPU avant entraînement: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

student_model.train()
global_step = 0
total_loss = 0

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch + 1}/{NUM_EPOCHS}")
    epoch_loss = 0
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}")
    
    for step, batch in enumerate(progress_bar):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        
        with torch.no_grad():
            teacher_outputs = teacher_model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            teacher_logits = teacher_outputs.logits.detach()
            del teacher_outputs
        
        with torch.amp.autocast('cuda'):
            student_outputs = student_model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            student_logits = student_outputs.logits
            
            loss = distillation_criterion(
                student_logits,
                teacher_logits,
                attention_mask
            )
            
            loss = loss / GRADIENT_ACCUMULATION_STEPS
        
        del teacher_logits, student_logits, student_outputs
        
        loss.backward()
        
        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            torch.nn.utils.clip_grad_norm_(student_model.parameters(), 1.0)
            
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad(set_to_none=True)
            
            global_step += 1
            
            if global_step % 10 == 0:
                gc.collect()
                torch.cuda.empty_cache()
        
        current_loss = loss.item() * GRADIENT_ACCUMULATION_STEPS
        epoch_loss += current_loss
        total_loss += current_loss
        
        progress_bar.set_postfix({
            'loss': f"{current_loss:.4f}",
            'lr': f"{scheduler.get_last_lr()[0]:.2e}",
            'mem': f"{torch.cuda.memory_allocated() / 1e9:.1f}GB"
        })
        
        del loss, input_ids, attention_mask
    
    avg_epoch_loss = epoch_loss / len(train_dataloader)
    print(f"\nEpoch {epoch + 1} terminée - Loss moyenne: {avg_epoch_loss:.4f}")
    
    gc.collect()
    torch.cuda.empty_cache()

avg_total_loss = total_loss / (len(train_dataloader) * NUM_EPOCHS)
print(f"\n" + "="*60)
print(f"DISTILLATION TERMINÉE")
print(f"Loss moyenne globale: {avg_total_loss:.4f}")
print("="*60)


DÉBUT DE LA DISTILLATION

Mémoire GPU avant entraînement: 4.20 GB

📚 Epoch 1/3


Epoch 1:   0%|          | 0/17000 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


KeyboardInterrupt: 

## 11. Évaluation qualitative (Teacher vs Student)

In [ ]:
# Fonction de génération
def generate_response(model, tokenizer, prompt, max_new_tokens=100):
    model.eval()
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
   
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.pad_token_id
        )
   
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


test_prompts = [
    "### Instruction:\nExplain the difference between supervised and unsupervised learning in simple terms.\n\n### Response:\n",
    "### Instruction:\nWrite a short email to your boss explaining that you will be late to work because of a doctor's appointment.\n\n### Response:\n",
    "### Instruction:\nGive me 5 creative ideas for a science fair project for a 10-year-old child.\n\n### Response:\n",
    "### Instruction:\nClassify the following animals as mammal, bird, reptile, or fish: dolphin, penguin, crocodile, salmon, bat.\n\n### Response:\n",
    "### Instruction:\nTranslate the following sentence into French: \"The quick brown fox jumps over the lazy dog.\"\n\n### Response:\n",
    "### Instruction:\nJanet has 8 apples. She gives 3 to her friend and then buys 5 more. How many apples does she have now?\n\n### Response:\n",
    "### Instruction:\nA store has 20 boxes of pencils. Each box contains 12 pencils. If they sell 15 boxes, how many pencils are left in the store?\n\n### Response:\n",
    "### Instruction:\nJohn has 5 bags of marbles. Each bag has 8 marbles. He gives away 18 marbles to his friends. How many marbles does he have left?\n\n### Response:\n",
    "### Instruction:\nA class has 30 students. 40% of them are girls. How many boys are in the class?\n\n### Response:\n",
    "### Instruction:\nWhy is it important to recycle plastic? Give at least 3 reasons.\n\n### Response:\n",
]


print("\n" + "="*100)
print("COMPARAISON QUALITATIVE : TEACHER vs STUDENT")
print("10 tests inspirés d'Alpaca et GSM8K")
print("="*100 + "\n")

for i, prompt in enumerate(test_prompts, 1):
    print(f"\n{'='*100}")
    print(f"Test {i:2d}/10")
    print(f"{'='*100}")
    print(f"\nPROMPT:\n{prompt.strip()}")
   
    print(f"\nTEACHER (Mistral-7B + LoRA):")
    teacher_response = generate_response(teacher_model, teacher_tokenizer, prompt, max_new_tokens=150)
    print(teacher_response[len(prompt):].strip())
   
    print(f"\nSTUDENT (Distilled TinyLLaMA):")
    student_response = generate_response(student_model, student_tokenizer, prompt, max_new_tokens=150)
    print(student_response[len(prompt):].strip())
    
    print("\n" + "-"*100)


COMPARAISON QUALITATIVE : TEACHER vs STUDENT
10 tests inspirés d'Alpaca et GSM8K


Test  1/10

📝 PROMPT:
### Instruction:
Explain the difference between supervised and unsupervised learning in simple terms.

### Response:

🎓 TEACHER (Mistral-7B + LoRA):
Supervised learning is when a machine is trained to identify patterns in a set of data by using labeled data. Unsupervised learning is when a machine is trained to identify patterns in data without any labels. Supervised learning is used to classify data, while unsupervised learning is used to identify patterns and group data. In supervised learning, the machine is trained to recognize a certain input and produce a certain output. In unsupervised learning, the machine is trained to recognize patterns in data and group them together. Supervised learning is more commonly used in tasks such as image classification, while unsupervised learning is more commonly used in tasks such as clustering and dimensionality reduction. Supervised learni

## 12. Sauvegarde du modèle distillé

In [ ]:
# Sauvegarder le modèle student distillé
print(f"Sauvegarde du modèle student dans {OUTPUT_MODEL_DIR}/...")

student_model.save_pretrained(OUTPUT_MODEL_DIR)
student_tokenizer.save_pretrained(OUTPUT_MODEL_DIR)

print("\nModèle distillé sauvegardé avec succès!")
print(f"\nPour charger le modèle:")
print(f"```python")
print(f"from transformers import AutoModelForCausalLM, AutoTokenizer")
print(f"")
print(f"model = AutoModelForCausalLM.from_pretrained('{OUTPUT_MODEL_DIR}')")
print(f"tokenizer = AutoTokenizer.from_pretrained('{OUTPUT_MODEL_DIR}')")
print(f"```")

## 13. Statistiques du modèle

In [ ]:
# Comparer les tailles des modèles
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

teacher_params = count_parameters(teacher_model)
student_params = count_parameters(student_model)
compression_ratio = teacher_params / student_params

print("\n" + "="*60)
print("STATISTIQUES DES MODÈLES")
print("="*60)
print(f"\nTeacher (Mistral-7B):")
print(f"   Paramètres: {teacher_params:,} ({teacher_params/1e9:.2f}B)")
print(f"\nStudent (TinyLLaMA):")
print(f"   Paramètres: {student_params:,} ({student_params/1e9:.2f}B)")
print(f"\nRatio de compression: {compression_ratio:.2f}x")
print(f"   Le student est {compression_ratio:.2f}x plus petit que le teacher")
print("\n" + "="*60)


STATISTIQUES DES MODÈLES

🎓 Teacher (Mistral-7B):
   Paramètres: 3,919,843,328 (3.92B)

🎯 Student (TinyLLaMA):
   Paramètres: 1,100,048,384 (1.10B)

📊 Ratio de compression: 3.56x
   Le student est 3.56x plus petit que le teacher



## Conclusion

La distillation de connaissances a permis de transférer les capacités du modèle teacher (Mistral-7B fine-tuné avec QLoRA) vers un modèle student plus compact (TinyLLaMA-1.1B).

Points clés:
- Le modèle student est environ 6-7x plus petit
- Inference plus rapide
- Empreinte mémoire réduite
- Performance raisonnable par rapport au teacher

Prochaines étapes possibles:
- Évaluation quantitative sur des benchmarks
- Fine-tuning additionnel du student sur des tâches spécifiques
- Optimisation pour le déploiement (quantization, ONNX, etc.)
- Tests de performance en production